In [25]:
import pyspark
from delta import *
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window
spark = (SparkSession.builder.appName("SparkSample").getOrCreate())

Read in a single file and make duplicates

In [26]:
window = Window.orderBy("value")

In [29]:
df_1 = (spark.read.text("textfile.txt")
        .withColumn("rn", F.row_number().over(window))
     )
df = df_1.unionAll(df_1).orderBy(F.col("rn"))
df.show(10)

+-----+---+
|value| rn|
+-----+---+
|     |  1|
|     |  1|
|     |  2|
|     |  2|
|     |  3|
|     |  3|
|     |  4|
|     |  4|
|     |  5|
|     |  5|
+-----+---+
only showing top 10 rows



25/02/12 02:40:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/12 02:40:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/12 02:40:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/12 02:40:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/12 02:40:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/12 02:40:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/12 0

method #1

In [31]:
df.distinct().orderBy("rn").show(10)

+-----+---+
|value| rn|
+-----+---+
|     |  1|
|     |  2|
|     |  3|
|     |  4|
|     |  5|
|     |  6|
|     |  7|
|     |  8|
|     |  9|
|     | 10|
+-----+---+
only showing top 10 rows



25/02/12 02:41:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/12 02:41:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/12 02:41:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/12 02:41:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/12 02:41:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/12 02:41:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/12 0

method 2

In [32]:
df.groupby("rn").show()

AttributeError: 'GroupedData' object has no attribute 'show'